In [1]:
!pip install lxml
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library

CLIENT_ID = 'XGF5F1WVM0W1G3WDCA4MY2PFPMGPC5SRVZNDKKNHLPWSYBKJ' # your Foursquare ID
CLIENT_SECRET = 'TSL2XDNGRPT5PECDN0JPIBIZVZ4UB1QLHJNVYCC52AJZRM2L' # your Foursquare Secret
VERSION = '20200725'
LIMIT = 30
print('libraries loaded')

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.8.2
  latest version: 4.8.3

Please update conda by running

    $ conda update -n base conda



# All requested packages already installed.

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.8.2
  latest version: 4.8.3

Please update conda by running

    $ conda update -n base conda



# All requested packages already installed.

libraries loaded


In [2]:
# read postal codes from web page into a data frame
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
lst_pstl_cd = pd.read_html(url)
df_pstl_cd = lst_pstl_cd[0]
df_pstl_cd.head()

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [3]:
# data pre processing 
# removing records with Borough Not Assigned
df_pstl_cd = df_pstl_cd[df_pstl_cd.Borough != "Not assigned"]
df_pstl_cd.head()

,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [4]:
df_pstl_cd.shape

(103, 3)

In [5]:
df_geo_cd=pd.read_csv("https://ibm.box.com/shared/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv")
df_geo_cd.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [6]:
# fetch lat,Lon for postal codes
df_final = pd.merge(df_pstl_cd,df_geo_cd,on = 'Postal Code')
df_final.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


In [7]:
# fetch Lat , Lon of Toronto city 
address = 'Toronto'
geolocator = Nominatim(user_agent="tt_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))


The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [8]:
# Display neighbourhood in toronto
# add markers to map
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

for lat, lng, borough, neighborhood in zip(df_final['Latitude'], df_final['Longitude'], df_final['Borough'], df_final['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [9]:

neighborhood_latitude = df_final['Latitude'] # neighborhood latitude value
neighborhood_longitude = df_final['Longitude']


In [14]:
# creating new data frame for resturants
column_names = ['neig long','neigh lat','venue name','venue lat','venue long','venu category']
#column_names = ['neig long','neigh lat','venue name']

df_venue = pd.DataFrame(columns = column_names)
venues_list = []
LIMIT = 100 
radius = 500
# call foursquare api for each neigh lat & long
for name, lat , long  in zip(df_final['Neighbourhood'],df_final['Latitude'],df_final['Longitude']):
    
    url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    lat, # neighbourhood latitude
    long, # neighbourhood longitude
    radius, 
    LIMIT)
    #venue_result = requests.get(url).json()
    venue_result = requests.get(url).json()["response"]['groups'][0]['items']

    # creat a list of relevant information of the venues
    
    venues_list.append([(name, 
                        lat, 
                        long, 
                        v['venue']['name'],
                        v['venue']['location']['lat'],
                        v['venue']['location']['lng'],
                        v['venue']['categories'][0]['name']) 
                       for v in venue_result])
    #print(venue_list[0])
    #for item in venues_list:
     # venue_list_item = pd.DataFrame(item)
      #df_venue.append(venue_list_item)
    
    #venue_list_item = pd.DataFrame(venue_list,columns = column_names)
    
    #df_venue = pd.concat([df_venue,venue_list_item])
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
  
    
nearby_venues.head()

    


{'meta': {'code': 429,
  'errorType': 'quota_exceeded',
  'errorDetail': 'Quota exceeded',
  'requestId': '5f1c371614f23a1bc5d4eaf1'},
 'response': {}}

In [ ]:
nearby_venues.groupby('Venue Category').count()

In [ ]:

from sklearn.cluster import KMeans

In [ ]:

labels=['Latitude','Longitude']

scaled=StandardScaler().fit_transform(nearby_venues[labels])
kmeans = KMeans(n_clusters=6, random_state=0).fit(scaled)
Restu['Labels']=kmeans.labels_
import matplotlib.cm as cm
import matplotlib.colors as colors
kclusters=6
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]